In [ ]:
from copy import *
from random import *
from jogos import *
from utils import *
import string

from collections import namedtuple

stateBreaktrhu = namedtuple('EstadoBreakthru', 'to_move whites blacks')

class EstadoBreakthru(stateBreaktrhu):

    def init(self, to_move, n=8):
        self.board = [['.' for i in range(n)] for i in range(n)]

        for i in range(2):
            for j in range(8):
                self.board[i][j] = 'W'

        for i in range(6, 8):
            for j in range(8):
                self.board[i][j] = 'B'

    


## CLASS JOGO ##
class JogoBT_16(Game):
    """Play Breakthrough on an 8 x 8 board, with Max (first player) playing 'W' (1).
    """
    def init(self, to_move, n=8):
        self.initial = EstadoBreakthru(to_move=1, n=8)

    def actions(self, state):
        "Legal moves for B and W"
        #Apenas queremos ver as ações possíveis para o jogador atual (to_move)
        if state.to_move == 1:
            for pos_w in state.whites:

        else:
            #TODO

    def result(self, state, move):
        pass

    def terminal_test(self, state):
        "A state is terminal if the whites reached line 8 or some black reached line 1."
        pass

    def display(self, state):
        pass

    def executa(self, estado, listaJogadas):
        "executa varias jogadas sobre um estado dado"
        "devolve o estado final "
        s = estado
        for j in listaJogadas:
            s = self.result(s, j)
        return s